In [1]:
# !pip install pytorch_lightning --quiet
# !pip install ray[tune] --quiet
!pip install transformers --quiet
!pip install "ray[tune]" torch torchvision pytorch-lightning --quiet

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

rootDir = '/content/drive/MyDrive/'

TIMESTAMP = '2022_03_30'

dataPath = rootDir + 'TP2/Datasets/Recipe1M/' + TIMESTAMP
tarPath = rootDir + 'Colab Notebooks/recime/data/' + TIMESTAMP
weightDir = rootDir + 'Colab Notebooks/recime/weights/'
logDir = rootDir + 'Colab Notebooks/recime/logs/titleTrainer/'
rayDir = rootDir + 'Colab Notebooks/recime/ray/titleTrainer/'


import sys
sys.path.append(rootDir + 'Colab Notebooks/recime/LSTM')

ModuleNotFoundError: No module named 'google.colab'

In [2]:
import sys
sys.path.append('D:/02_Studium/SBX/mad-recime/network/LSTM')

tarPath = 'D:/02_Studium/SBX/mad-recime/data/2022_03_19'
logDir = 'D:/02_Studium/SBX/mad-recime/network/LSTM/logs/pl/'
rayDir = 'D:/02_Studium/SBX/mad-recime/network/LSTM/ray/'

In [3]:
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import Callback
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.utilities.seed import seed_everything
from pytorch_lightning import loggers as pl_loggers

import torch
import numpy as np
import re
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
from ray.tune.integration.pytorch_lightning import TuneReportCallback,  TuneReportCheckpointCallback



In [4]:
import src.nlp_utils.model as models
import src.nlp_utils.data_module as data_module

import importlib

In [5]:
available_gpus = [torch.cuda.device(i) for i in range(torch.cuda.device_count())]
num_gpus = len(available_gpus)
num_gpus

0

In [6]:
# set seed to get consistent results, deactivate if random results are wanted
seed_everything(42)

Global seed set to 42


42

In [8]:
importlib.reload(data_module)

titleMod = data_module.TitleDataModule(tarPath, setSize=8)
titleMod.setup()

D:\02_Studium/SBX/mad-recime/network/LSTM\src\nlp_utils\preProc.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  baseFrame = baseFrame.append(pd.read_pickle(dataSetSplits[split]))
D:\02_Studium/SBX/mad-recime/network/LSTM\src\nlp_utils\preProc.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  baseFrame = baseFrame.append(pd.read_pickle(dataSetSplits[split]))


In [12]:
importlib.reload(models)

from src.nlp_utils.config import create_config

tb_logger = pl_loggers.TensorBoardLogger(save_dir=logDir)

config = create_config({})

config['vocabSize'] = titleMod.vocab_size

model = models.EmbedLSTM(config)

In [13]:
trainer = Trainer(gpus=num_gpus, max_epochs=config['epochs'], logger=tb_logger)
trainer.fit(model, titleMod)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
C:\Users\Hannes\miniconda3\envs\madEnv\lib\site-packages\pytorch_lightning\core\datamodule.py:469: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  rank_zero_deprecation(

  | Name      | Type             | Params
-----------------------------------------------
0 | embed     | Embedding        | 171 K 
1 | lstm      | LSTM             | 571 K 
2 | linear    | Linear           | 147 K 
3 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
889 K     Trainable params
0         Non-trainable params
889 K     Total params
3.560     Total estimated model params size (MB)


Global seed set to 42
C:\Users\Hannes\miniconda3\envs\madEnv\lib\site-packages\pytorch_lightning\trainer\data_loading.py:432: UserWarning: The number of training samples (7) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Epoch 0:   0%|          | 0/10 [00:00<?, ?it/s] 

# Test

In [ ]:
# mnist = MNISTDataModule(my_path)
# model = LitClassifier()

# trainer = Trainer()
# trainer.fit(model, mnist)

# Optimization

In [19]:
early_stop_callback = EarlyStopping(
    monitor="val_loss",
    min_delta=0.05,
    patience=3,
    verbose=False,
    mode="min",
    divergence_threshold=3.00,
)

class MyPrintingCallback(Callback):
    def on_fit_start(self, trainer, pl_module):
        print("Starting to train!")

    def on_fit_end(self, trainer, pl_module):
        print("Finished training")

    def on_test_start(self, trainer, pl_module):
        print("Start to test")

    def on_test_end(self, trainer, pl_module):
        print("Finished testing")


# checkpoint_callback = ModelCheckpoint(
#     monitor="val_epoch_stance_F1",
#     filename="{epoch}-{val_loss:.2f}-{val_epoch_stance_F1:.2f}",
#     save_top_k=3,
#     mode="max",
# )

callback = TuneReportCallback(
    {"loss": "val_loss"}, on="validation_end"
)


# training loop that tests out different hyperparameters and saves the results into the log folder
def train_tune(config, callbacks, epochs=100):#, gpus=num_gpus):
    dataModule = data_module.TitleDataModule(tarPath, setSize=8, num_workers=2, config=config)
    dataModule.setup()
    
    config["vocabSize"] = dataModule.vocab_size
    
    model = models.EmbedLSTM(config)

    trainer = pl.Trainer(
        # gpus=gpus,
        log_every_n_steps=1,
        flush_logs_every_n_steps=1,
        callbacks=callbacks,
        deterministic=True,
        default_root_dir=logDir,
        max_epochs=epochs,
    )  

    trainer.fit(model, datamodule=dataModule)

    # might not be called due to scheduler and reporter which cancel training early if results don't look promising
    # trainer.test(model, datamodule=dataModule)


# config with radom sampling for learning rate and batch size
config = {
    'lr': tune.sample_from(lambda: 10 ** np.random.uniform(-5, -3)),
    'batchSize': tune.choice([1, 8, 16, 32, 64, 128]),
    'hiddenSize': tune.choice([16, 32, 64, 128, 256]),
    'epochs': 30,
    'num_trials': 5,
}


# callbacks = [MyPrintingCallback(), early_stop_callback]
callbacks = [callback]

scheduler = ASHAScheduler(max_t=config['epochs'], grace_period=1, reduction_factor=2)

reporter = CLIReporter(
    parameter_columns=['lr', 'batchSize', 'hiddenSize'],
    metric_columns=['loss'],
)

# ray.init(local_mode=True, num_cpus=4, num_gpus=0)  # for debugging

# create versioning for multiple runs
# def atoi(text):
#     return int(text) if text.isdigit() else text


# def natural_keys(text):
#     return [atoi(c) for c in re.split("(\d+)", text)]


# log_dir = "../logs/StancePrediction_SemEval/lightning_logs/"
# log_path = os.path.join(path, log_dir)
# os.makedirs(os.path.dirname(log_path), exist_ok=True)
# ver = os.listdir(os.path.join(path, log_dir))
# ver.sort(key=natural_keys)
# if ver:
#     version = int(ver[-1].split("_", 2)[-1]) + 1
# else:
#     version = 0

In [20]:
# start hyperparameter optimization
analysis = tune.run(
    tune.with_parameters(
        train_tune, callbacks=callbacks, epochs=1#, gpus=0
    ),
    config=config,
    num_samples=config['num_trials'],
    local_dir=rayDir,
    name='version_ReciMe',
    metric='loss',
    mode='min',
    scheduler=scheduler,
    progress_reporter=reporter,
)

# get some information from the optimization
best_trial = analysis.best_trial  # Get best trial
best_config = analysis.best_config  # Get best trial's hyperparameters
best_logdir = analysis.best_logdir  # Get best trial's logdir
best_checkpoint = analysis.best_checkpoint  # Get best trial's best checkpoint
best_result = analysis.best_result  # Get best trial's last results
best_result_df = analysis.best_result_df  # Get best result as pandas dataframe

# Get a dataframe with the last results for each trial
df_results = analysis.results_df

# Get a dataframe of results for a specific score or mode
df = analysis.dataframe(metric="loss", mode="min")
# df2 = analysis.dataframe(metric="val_epoch_F1", mode="max") # check how to include multiple metrics


print("Best hyperparameters found were: ", analysis.best_config)

2022-04-01 22:51:01,920	WARNING tune.py:583 -- Tune detects GPUs, but no trials are using GPUs. To enable trials to use GPUs, set tune.run(resources_per_trial={'gpu': 1}...) which allows Tune to expose 1 GPU to each trial. You can also override `Trainable.default_resource_request` if using the Trainable API.


== Status ==
Current time: 2022-04-01 22:51:02 (running for 00:00:00.62)
Memory usage on this node: 7.2/7.9 GiB: ***LOW MEMORY*** less than 10% of the memory on this node is available for use. This can cause unexpected crashes. Consider reducing the memory used by your application or reducing the Ray object store size by setting `object_store_memory` when calling `ray.init`.
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 16.000: None | Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 0/4 CPUs, 0/1 GPUs, 0.0/0.6 GiB heap, 0.0/0.3 GiB objects
Result logdir: D:\02_Studium\SBX\mad-recime\network\LSTM\ray\version_ReciMe
Number of trials: 5/5 (5 PENDING)
+------------------------+----------+-------+-------------+-------------+--------------+
| Trial name             | status   | loc   |          lr |   batchSize |   hiddenSize |
|------------------------+----------+-------+-------------+-------------+--------------|
| train_tune_70cbe_00000 | 

2022-04-01 22:51:17,089	WARNING util.py:163 -- The `start_trial` operation took 0.596 s, which may be a performance bottleneck.
2022-04-01 22:51:23,520	WARNING worker.py:1326 -- Traceback (most recent call last):
  File "python\ray\_raylet.pyx", line 633, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 674, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 640, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 644, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 593, in ray._raylet.execute_task.function_executor
  File "C:\Users\Hannes\miniconda3\envs\madEnv\lib\site-packages\ray\_private\function_manager.py", line 648, in actor_method_executor
    return method(__ray_actor, *args, **kwargs)
  File "C:\Users\Hannes\miniconda3\envs\madEnv\lib\site-packages\ray\_private\function_manager.py", line 544, in temporary_actor_method
    raise RuntimeError(
RuntimeError: The actor with name ImplicitFunc failed to import on the 

== Status ==
Current time: 2022-04-01 22:51:18 (running for 00:00:16.22)
Memory usage on this node: 7.4/7.9 GiB: ***LOW MEMORY*** less than 10% of the memory on this node is available for use. This can cause unexpected crashes. Consider reducing the memory used by your application or reducing the Ray object store size by setting `object_store_memory` when calling `ray.init`.
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 16.000: None | Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 1.0/4 CPUs, 0/1 GPUs, 0.0/0.6 GiB heap, 0.0/0.3 GiB objects
Result logdir: D:\02_Studium\SBX\mad-recime\network\LSTM\ray\version_ReciMe
Number of trials: 5/5 (4 PENDING, 1 RUNNING)
+------------------------+----------+-------+-------------+-------------+--------------+
| Trial name             | status   | loc   |          lr |   batchSize |   hiddenSize |
|------------------------+----------+-------+-------------+-------------+--------------|
| train_tune_7

2022-04-01 22:51:24,554	WARNING util.py:163 -- The `on_step_begin` operation took 0.515 s, which may be a performance bottleneck.
2022-04-01 22:51:24,706	ERROR trial_runner.py:920 -- Trial train_tune_70cbe_00000: Error processing event.
Traceback (most recent call last):
  File "C:\Users\Hannes\miniconda3\envs\madEnv\lib\site-packages\ray\tune\trial_runner.py", line 886, in _process_trial
    results = self.trial_executor.fetch_result(trial)
  File "C:\Users\Hannes\miniconda3\envs\madEnv\lib\site-packages\ray\tune\ray_trial_executor.py", line 675, in fetch_result
    result = ray.get(trial_future[0], timeout=DEFAULT_GET_TIMEOUT)
  File "C:\Users\Hannes\miniconda3\envs\madEnv\lib\site-packages\ray\_private\client_mode_hook.py", line 105, in wrapper
    return func(*args, **kwargs)
  File "C:\Users\Hannes\miniconda3\envs\madEnv\lib\site-packages\ray\worker.py", line 1765, in get
    raise value
ray.exceptions.RayActorError: The actor died unexpectedly before finishing this task.
	class_n

...

== Status ==
Current time: 2022-04-01 22:52:50 (running for 00:01:48.86)
Memory usage on this node: 7.3/7.9 GiB: ***LOW MEMORY*** less than 10% of the memory on this node is available for use. This can cause unexpected crashes. Consider reducing the memory used by your application or reducing the Ray object store size by setting `object_store_memory` when calling `ray.init`.
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 16.000: None | Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 3.0/4 CPUs, 0/1 GPUs, 0.0/0.6 GiB heap, 0.0/0.3 GiB objects
Result logdir: D:\02_Studium\SBX\mad-recime\network\LSTM\ray\version_ReciMe
Number of trials: 5/5 (2 ERROR, 3 RUNNING)
+------------------------+----------+-------+-------------+-------------+--------------+
| Trial name             | status   | loc   |          lr |   batchSize |   hiddenSize |
|------------------------+----------+-------+-------------+-------------+--------------|
| train_tune_70c

2022-04-01 22:52:51,117	ERROR trial_runner.py:920 -- Trial train_tune_70cbe_00001: Error processing event.
Traceback (most recent call last):
  File "C:\Users\Hannes\miniconda3\envs\madEnv\lib\site-packages\ray\tune\trial_runner.py", line 886, in _process_trial
    results = self.trial_executor.fetch_result(trial)
  File "C:\Users\Hannes\miniconda3\envs\madEnv\lib\site-packages\ray\tune\ray_trial_executor.py", line 675, in fetch_result
    result = ray.get(trial_future[0], timeout=DEFAULT_GET_TIMEOUT)
  File "C:\Users\Hannes\miniconda3\envs\madEnv\lib\site-packages\ray\_private\client_mode_hook.py", line 105, in wrapper
    return func(*args, **kwargs)
  File "C:\Users\Hannes\miniconda3\envs\madEnv\lib\site-packages\ray\worker.py", line 1765, in get
    raise value
ray.exceptions.RayActorError: The actor died unexpectedly before finishing this task.
	class_name: ImplicitFunc
	actor_id: a62c0cfd1d38d939e76affed01000000
	pid: 16256
	namespace: dfa27098-ba22-48a7-a645-ed4d5ff7d961
	ip: 12

Result for train_tune_70cbe_00001:
  trial_id: 70cbe_00001
  
== Status ==
Current time: 2022-04-01 22:52:51 (running for 00:01:49.30)
Memory usage on this node: 7.3/7.9 GiB: ***LOW MEMORY*** less than 10% of the memory on this node is available for use. This can cause unexpected crashes. Consider reducing the memory used by your application or reducing the Ray object store size by setting `object_store_memory` when calling `ray.init`.
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 16.000: None | Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 0/4 CPUs, 0/1 GPUs, 0.0/0.6 GiB heap, 0.0/0.3 GiB objects
Result logdir: D:\02_Studium\SBX\mad-recime\network\LSTM\ray\version_ReciMe
Number of trials: 5/5 (5 ERROR)
+------------------------+----------+-------+-------------+-------------+--------------+
| Trial name             | status   | loc   |          lr |   batchSize |   hiddenSize |
|------------------------+----------+-------+-----------

TuneError: ('Trials did not complete', [train_tune_70cbe_00000, train_tune_70cbe_00001, train_tune_70cbe_00002, train_tune_70cbe_00003, train_tune_70cbe_00004])

2022-04-02 11:12:43,035	WARNING worker.py:1326 -- The node with node id: fb972aa1896eeec4945941e547fa10f3a552dffd6291b1f357977c0b and ip: 127.0.0.1 has been marked dead because the detector has missed too many heartbeats from it. This can happen when a raylet crashes unexpectedly or has lagging heartbeats.
